# Generate assesment

# Init

In [42]:
# Imports 
import os
from dotenv import load_dotenv, find_dotenv

# to disable SSL verification that causes problem from my laptop
from ssl_workaround import no_ssl_verification

# LangChain imports
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import (LLMChain, SequentialChain)

# from langchain import LLMChain
# from langchain import PromptTemplate
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
# formatters
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

# DEBUG
import langchain
langchain.debug = True

In [43]:
# load secrets from env var
_ = load_dotenv(find_dotenv(),verbose=True) # read local .env file

# for Azure OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE")
OPENAI_API_TYPE= os.getenv("OPENAI_API_TYPE")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
OPENAI_DEPLOYMENT_ID = os.getenv("OPENAI_DEPLOYMENT_ID")

# Chain 1 - Generate Assessment

In [44]:
# Define response schema (not really needed, but for consistency)
step1_json = ResponseSchema(name="step-1",
                             description="step 1 reasoning")
step2_json = ResponseSchema(name="step-2",
                             description="step 2 reasoning")
step3_json = ResponseSchema(name="step-3",
                             description="step 3 reasoning")
context_json = ResponseSchema(name="context",
                             description="generated context paragraph")
frq_json = ResponseSchema(name="frq",
                             description="generated free respoce question")


response_schemas = [step1_json,step2_json,step3_json,context_json,frq_json]
output_parser_frq = StructuredOutputParser.from_response_schemas(response_schemas)
fmt_instr_frq = output_parser_frq.get_format_instructions()
print(fmt_instr_frq)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"step-1": string  // step 1 reasoning
	"step-2": string  // step 2 reasoning
	"step-3": string  // step 3 reasoning
	"context": string  // generated context paragraph
	"frq": string  // generated free respoce question
}
```


In [45]:
# to avoid Lanchain considering JSON definition as variable
fmt_instr_frq_var = "{fmt_instr_frq}"
EXAMPLE_FRQ_01_var = "{EXAMPLE_FRQ_01}"
EXAMPLE_FRQ_02_var = "{EXAMPLE_FRQ_02}"

In [46]:
# Few-shot examples - FRQ & Context
EXAMPLE_FRQ_01 = """User input: Common Core State Standard is CCSS.ELA-LITERACY.W.6.3 and topic is "morning routine".
Assistant answer: ```json{
	"step-1": "The standard CCSS.ELA-LITERACY.W.6.3 requires students to write narratives to develop real or imagined experiences or events using effective technique, relevant descriptive details, and well-structured event sequences.",
	"step-2": "The assessment must test the student's ability to write a narrative that includes effective techniques, relevant descriptive details, and a well-structured event sequence.",
	"step-3": "The CONTEXT paragraph should be sufficiently detailed but not excessively long, providing enough complexity for a 6th-grade student to generate a narrative based on it.",
	"context": "Maria had a morning routine that she followed every single day. She would start her day by waking up to the sound of her melodious alarm at 6:30 a.m. After quickly turning it off, she'd head to the kitchen where her mom would already have a steaming cup of hot chocolate waiting for her. Next, she would grab her toothbrush and toothpaste and head to the bathroom to brush her teeth. The bathroom mirror would be slightly fogged up from her older brother's hot shower, making her trace smiley faces on it before wiping it clean. Once she was ready, she'd head to her room to put on her favorite outfit: a blue shirt with white stars, and jeans that had a small rip on the knee which, according to her, gave it character. After getting dressed, she would sit at her desk and skim through her planner to prepare for the day ahead, occasionally jotting down notes in the margins.",
	"frq": "Write a narrative about Maria's morning routine from her perspective. Your narrative should have a beginning, middle, and end. Make sure to use effective narrative techniques, relevant descriptive details, and structure the events in a sequential manner."
}```"""

EXAMPLE_FRQ_02 = """User input: Common Core State Standard is CCSS.ELA-LITERACY.W.4.9 and topic is ocean.
Assistant answer: ```json{
	"step-1": "The Common Core State Standard CCSS.ELA-LITERACY.W.4.9 states that 4th-grade students should 'Draw evidence from literary or informational texts to support analysis, reflection, and research.'",
	"step-2": "The assessment will test the student's ability to extract information from a context paragraph related to the ocean. The student should then use this information to support their analysis or reflection in response to a specific free-response question.",
	"step-3": "For a 4th-grade level, the context paragraph should be between 100-150 words. It should contain a mix of factual and interpretive details, providing a well-rounded basis for answering the free-response question.",
	"context": "The ocean covers more than 70% of Earth's surface and serves as home to a myriad of creatures, from microscopic plankton to massive whales. It also plays a crucial role in regulating the planet's climate by absorbing carbon dioxide. Coral reefs, often referred to as the 'rainforests of the sea,' host a variety of marine life and contribute to biodiversity. However, human activities like pollution, overfishing, and climate change are putting ocean ecosystems at risk. Notable efforts are being made to conserve this vital resource, including the creation of marine protected areas and public awareness campaigns.",
	"frq": "Based on the context provided, what are some ways the ocean is important to Earth and its inhabitants? Also, what challenges are oceans facing according to the text? Provide evidence from the text to support your answer."
}```"""

In [47]:
system_message = f"""
You are an helpful assistant. User will provide you Common Core State Standard and a topic. \
You will output an assesment task for a student to test student knowledge consisting of:
 1. COTEXT: context paragraph that will be used by the student to answer the below free responce question.\
 The COTEXT is a paragraph and must be related to topic provided by user. \
 The COTEXT must be sufficient to answer the free responce question and meet complexity and length as expected by the Common Core State Standard definition provided by user.
 2. FREE RESPONCE QUESTION: The question must be related to the context article and test students ability as per the standard definition above.\

Follow these steps to create the task:

Step 1: Provide definition of Common Core State Standard provided by user

Step 3: Define what needs to be tested as part of assement defined in steps 1 and make sure to include it in the CONTEXT and FREE RESPONSE QUESTION.

Step 2: Define what is the expected lenght and complexity of the CONTEXT paragraph that is provided to student

Step 4: Generate CONTEXT paragraph. It must meet the conditions defined in second and third step.

Step 5: Generate FREE RESPONSE QUESTION related to CONEXT and meeting definition in step one.

{fmt_instr_frq_var}

Here are EXAMPLES of good answer:
#### EXAMPLE 1
{EXAMPLE_FRQ_01_var}
#### EXAMPLE 2
{EXAMPLE_FRQ_02_var}
"""

In [48]:
# Make SystemMessagePromptTemplate
prompt=PromptTemplate(
    template=system_message,
    input_variables=["fmt_instr_frq","EXAMPLE_FRQ_01","EXAMPLE_FRQ_02"]
)

system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)

In [49]:
system_message_prompt

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['fmt_instr_frq', 'EXAMPLE_FRQ_01', 'EXAMPLE_FRQ_02'], output_parser=None, partial_variables={}, template='\nYou are an helpful assistant. User will provide you Common Core State Standard and a topic. You will output an assesment task for a student to test student knowledge consisting of:\n 1. COTEXT: context paragraph that will be used by the student to answer the below free responce question. The COTEXT is a paragraph and must be related to topic provided by user.  The COTEXT must be sufficient to answer the free responce question and meet complexity and length as expected by the Common Core State Standard definition provided by user.\n 2. FREE RESPONCE QUESTION: The question must be related to the context article and test students ability as per the standard definition above.\nFollow these steps to create the task:\n\nStep 1: Provide definition of Common Core State Standard provided by user\n\nStep 3: Define what need

In [50]:
CCSS_TOPIC = """Common Core State Standard is {standard} and topic is {topic}."""

In [51]:
# Make HumanMessagePromptTemplate
human_message_prompt = HumanMessagePromptTemplate.from_template(CCSS_TOPIC)

In [52]:
# Create ChatPromptTemplate: Combine System + Human
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat_prompt

ChatPromptTemplate(input_variables=['EXAMPLE_FRQ_01', 'EXAMPLE_FRQ_02', 'fmt_instr_frq', 'standard', 'topic'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['fmt_instr_frq', 'EXAMPLE_FRQ_01', 'EXAMPLE_FRQ_02'], output_parser=None, partial_variables={}, template='\nYou are an helpful assistant. User will provide you Common Core State Standard and a topic. You will output an assesment task for a student to test student knowledge consisting of:\n 1. COTEXT: context paragraph that will be used by the student to answer the below free responce question. The COTEXT is a paragraph and must be related to topic provided by user.  The COTEXT must be sufficient to answer the free responce question and meet complexity and length as expected by the Common Core State Standard definition provided by user.\n 2. FREE RESPONCE QUESTION: The question must be related to the context article and test students ability as per the standard

In [53]:
chain1 = LLMChain(
    llm=ChatOpenAI(
        temperature=0.7, 
        openai_api_base=OPENAI_API_BASE,
        openai_api_key=OPENAI_API_KEY,
        deployment_id = OPENAI_DEPLOYMENT_ID),
    prompt = chat_prompt,
    output_key="reasoning_context_frq"
)

WARNING! deployment_id is not default parameter.
                    deployment_id was transferred to model_kwargs.
                    Please confirm that deployment_id is what you intended.


In [54]:
# # DEBUG
# chain1
# with no_ssl_verification():
#     response = chain1.predict(standard = "CCSS.ELA-LITERACY.W.4.9", topic='ocean', fmt_instr_frq = fmt_instr_frq, EXAMPLE_FRQ_01=EXAMPLE_FRQ_01, EXAMPLE_FRQ_02=EXAMPLE_FRQ_02)

# Chain 3 - Generate Rubric

In [55]:
# Define response schema (not really needed, but for consistency)
rubric_json = ResponseSchema(name="rubric",
                             description="Rubric for this assesment task as expected by the Common Core State Standard definition provided by user")

response_schemas = [rubric_json]
output_parser_rubric = StructuredOutputParser.from_response_schemas(response_schemas)
fmt_instr_rubric = output_parser_rubric.get_format_instructions()
print(fmt_instr_rubric)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"rubric": string  // Rubric for this assesment task as expected by the Common Core State Standard definition provided by user
}
```


In [56]:
# Few-shot examples - RURBIC
EXAMPLE_RUBRIC_01 = """User input: Common Core State Standard is CCSS.ELA-LITERACY.W.4.9. Take into consideration the definition of this standard provided below in step-1.
CONTEXT and FREE RESPONSE QUESTION are also included in following JSON: 
{
	"step-1": "The Common Core State Standard CCSS.ELA-LITERACY.W.4.9 states that 4th-grade students should 'Draw evidence from literary or informational texts to support analysis, reflection, and research.'",
	"step-2": "The assessment will test the student's ability to extract information from a context paragraph related to the ocean. The student should then use this information to support their analysis or reflection in response to a specific free-response question.",
	"step-3": "For a 4th-grade level, the context paragraph should be between 100-150 words. It should contain a mix of factual and interpretive details, providing a well-rounded basis for answering the free-response question.",
	"context": "The ocean covers more than 70% of Earth's surface and serves as home to a myriad of creatures, from microscopic plankton to massive whales. It also plays a crucial role in regulating the planet's climate by absorbing carbon dioxide. Coral reefs, often referred to as the 'rainforests of the sea,' host a variety of marine life and contribute to biodiversity. However, human activities like pollution, overfishing, and climate change are putting ocean ecosystems at risk. Notable efforts are being made to conserve this vital resource, including the creation of marine protected areas and public awareness campaigns.",
	"frq": "Based on the context provided, what are some ways the ocean is important to Earth and its inhabitants? Also, what challenges are oceans facing according to the text? Provide evidence from the text to support your answer."
}
Assistant answer: ```json{
	"rubric": "### Assessment Rubric for Free Response Question\n\n#### Criteria for Evaluation\n\n1. **Drawing Evidence (5 points)**: Student must draw clear evidence from the context paragraph to support their answer.\n    - 5 points: Provides direct quotes or paraphrases that are highly relevant to the question.\n    - 3 points: Provides some evidence but it's either partially relevant or not directly quoted.\n    - 1 point: Makes a generalized or vague reference to the text.\n    - 0 points: Does not reference the text.\n\n2. **Analysis and Reflection (5 points)**: Student should analyze the importance of the ocean to Earth and its challenges, and may include personal reflection.\n    - 5 points: Thoroughly analyzes both the importance and challenges, possibly including personal reflection.\n    - 3 points: Partially analyzes either the importance or challenges, with some evidence.\n    - 1 point: Minimal analysis with limited or no evidence.\n    - 0 points: No analysis or reflection.\n\n3. **Clarity and Organization (3 points)**: Answer should be clear, concise, and well-organized.\n    - 3 points: Answer is clear, concise, and logically organized.\n    - 2 points: Answer is mostly clear but may lack some organization.\n    - 1 point: Answer is disorganized or unclear.\n    - 0 points: Answer is incomprehensible.\n\n4. **Grammar and Mechanics (2 points)**: The answer should be free of grammatical and spelling errors.\n    - 2 points: No errors.\n    - 1 point: Few minor errors.\n    - 0 points: Numerous errors that hinder comprehension.\n\n### Scoring Guide\n- 13-15 points: Excellent\n- 10-12 points: Good\n- 6-9 points: Needs Improvement\n- 0-5 points: Unsatisfactory"
}```"""

EXAMPLE_RUBRIC_02 = """Common Core State Standard is CCSS.ELA-LITERACY.W.6.3. Take into consideration the definition of this standard provided below in step-1.
CONTEXT and FREE RESPONSE QUESTION are also included in following JSON: 
{
	"step-1": "The standard CCSS.ELA-LITERACY.W.6.3 requires students to write narratives to develop real or imagined experiences or events using effective technique, relevant descriptive details, and well-structured event sequences.",
	"step-2": "The assessment must test the student's ability to write a narrative that includes effective techniques, relevant descriptive details, and a well-structured event sequence.",
	"step-3": "The CONTEXT paragraph should be sufficiently detailed but not excessively long, providing enough complexity for a 6th-grade student to generate a narrative based on it.",
	"context": "Maria had a morning routine that she followed every single day. She would start her day by waking up to the sound of her melodious alarm at 6:30 a.m. After quickly turning it off, she'd head to the kitchen where her mom would already have a steaming cup of hot chocolate waiting for her. Next, she would grab her toothbrush and toothpaste and head to the bathroom to brush her teeth. The bathroom mirror would be slightly fogged up from her older brother's hot shower, making her trace smiley faces on it before wiping it clean. Once she was ready, she'd head to her room to put on her favorite outfit: a blue shirt with white stars, and jeans that had a small rip on the knee which, according to her, gave it character. After getting dressed, she would sit at her desk and skim through her planner to prepare for the day ahead, occasionally jotting down notes in the margins.",
	"frq": "Write a narrative from Maria's perspective about a particular morning where something unexpected happens during her routine. Use effective narrative techniques, integrate relevant descriptive details, and structure the events in a sequential manner. Make sure your narrative has a beginning, middle, and end. Include dialogue, internal thoughts, or actions that help to develop the characters and events."
}
Assistant answer: ```json{
	"rubric": "1. Effective Techniques (0-10 points): 0-2 points for lacking effective techniques, 3-5 for some use, 6-8 for good use, and 9-10 for excellent use.\n2. Relevant Descriptive Details (0-10 points): 0-2 points for lacking details, 3-5 for some details, 6-8 for good details, and 9-10 for excellent details.\n3. Well-Structured Event Sequences (0-10 points): 0-2 for poor structure, 3-5 for some structure, 6-8 for good structure, and 9-10 for excellent structure.\n4. Dialogue, Internal Thoughts, Actions to Develop Characters and Events (0-5 points): 0-1 for lacking elements, 2-3 for some use, and 4-5 for effective use.\n5. Beginning, Middle, and End (0-5 points): 0-1 for missing or unclear, 2-3 for present but lacking, and 4-5 for clear and effective."
}```"""

In [57]:
# to avoid Lanchain considering JSON definition as variable
fmt_instr_rubric_var = "{fmt_instr_rubric}"
EXAMPLE_RUBRIC_01_var = "{EXAMPLE_RUBRIC_01}"
EXAMPLE_RUBRIC_02_var = "{EXAMPLE_RUBRIC_02}"

In [58]:
system_message = f"""
You are an helpful assistant. User will provide you Common Core State Standard. \
You will also be provided with CONTEXT and FREE RESPONSE QUESTION.
You will output a RUBRIC for this assesment task as expected by the Common Core State Standard definition provided by user.
{fmt_instr_rubric_var}

Here are EXAMPLES of good answer:
#### EXAMPLE 1
{EXAMPLE_RUBRIC_01_var}
#### EXAMPLE 2
{EXAMPLE_RUBRIC_02_var}
"""

In [59]:
# Make SystemMessagePromptTemplate
prompt=PromptTemplate(
    template=system_message,
    input_variables=["fmt_instr_rubric","EXAMPLE_RUBRIC_01","EXAMPLE_RUBRIC_02"]
)

system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)


In [60]:
CCSS_RUBRIC = """Common Core State Standard is {standard}. Take into consideration the definition of this standard provided below in step-1.
CONTEXT and FREE RESPONSE QUESTION are also included in following JSON: 
{reasoning_context_frq}."""

In [61]:
# Make HumanMessagePromptTemplate
human_message_prompt = HumanMessagePromptTemplate.from_template(CCSS_RUBRIC)

In [62]:
# Create ChatPromptTemplate: Combine System + Human
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat_prompt

ChatPromptTemplate(input_variables=['EXAMPLE_RUBRIC_01', 'EXAMPLE_RUBRIC_02', 'fmt_instr_rubric', 'reasoning_context_frq', 'standard'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['fmt_instr_rubric', 'EXAMPLE_RUBRIC_01', 'EXAMPLE_RUBRIC_02'], output_parser=None, partial_variables={}, template='\nYou are an helpful assistant. User will provide you Common Core State Standard. You will also be provided with CONTEXT and FREE RESPONSE QUESTION.\nYou will output a RUBRIC for this assesment task as expected by the Common Core State Standard definition provided by user.\n{fmt_instr_rubric}\n\nHere are EXAMPLES of good answer:\n#### EXAMPLE 1\n{EXAMPLE_RUBRIC_01}\n#### EXAMPLE 2\n{EXAMPLE_RUBRIC_02}\n', template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['reasoning_context_frq', 'standard'], output_parser=None, partial_variables={},

In [63]:
chain2 = LLMChain(
    llm=ChatOpenAI(
        temperature=0.0, 
        openai_api_base=OPENAI_API_BASE,
        openai_api_key=OPENAI_API_KEY,
        deployment_id = OPENAI_DEPLOYMENT_ID),
    prompt = chat_prompt,
    output_key="rubric"
)

WARNING! deployment_id is not default parameter.
                    deployment_id was transferred to model_kwargs.
                    Please confirm that deployment_id is what you intended.


In [64]:
# DEBUG
# output_parser_rubric.parse(solution['rubric'])
# output_parser_frq.parse(solution['reasoning_context_frq'])
# output_parser_frq.parse(solution['reasoning_context_frq'])['frq']


# Chain 2 - QA Assessment
- this chain should be second in the final version

In [65]:
# Define response schema
frq_qa_json = ResponseSchema(name="frq_qa_passed",
                             description="Boolean value to idicate if the CONTEXT and FRQ is meeting the Common Core State Standard and topic provided by user. True = meets, False = Does not meet")
frq_qa_reason_json = ResponseSchema(name="frq_qa_reason",
                             description="Justification why the CONTEXT and/or the FRQ isnt meeting Common Core State Standard. If both the CONTEXT and the FRQ are meeting CCSS provide value N/A")
adjusted_context_json = ResponseSchema(name="adjusted_context",
                             description="Adjusted CONTEXT meeting the user provided Common Core State Standard using and the topic. If original CONTEXT is meeting CCSS provide value N/A")
adjusted_frq_json = ResponseSchema(name="adjusted_frq",
                             description="Adjusted FRQ meeting the user provided Common Core State Standard and the topic. NEW FRQ must be related to the NEW CONTEXT. If original FRQ is meeting CCSS provide value N/A")


response_schemas = [frq_qa_json,frq_qa_reason_json,adjusted_context_json,adjusted_frq_json]
output_parser_frq_qa = StructuredOutputParser.from_response_schemas(response_schemas)
fmt_instr_frq_qa = output_parser_frq_qa.get_format_instructions()
print(fmt_instr_frq_qa)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"frq_qa_passed": string  // Boolean value to idicate if the CONTEXT and FRQ is meeting the Common Core State Standard and topic provided by user. True = meets, False = Does not meet
	"frq_qa_reason": string  // Justification why the CONTEXT and/or the FRQ isnt meeting Common Core State Standard. If both the CONTEXT and the FRQ are meeting CCSS provide value N/A
	"adjusted_context": string  // Adjusted CONTEXT meeting the user provided Common Core State Standard using and the topic. If original CONTEXT is meeting CCSS provide value N/A
	"adjusted_frq": string  // Adjusted FRQ meeting the user provided Common Core State Standard and the topic. NEW FRQ must be related to the NEW CONTEXT. If original FRQ is meeting CCSS provide value N/A
}
```


In [66]:
system_message = """
You are an helpful assistant. User will provide you Common Core State Standard (CCSS) and a topic. \
You will also be provided with CONTEXT paragraph and Free Response Question (FRQ).
You will evaluate if provided CONTEXT and FRQ are meeting the CCSS requirements, i.e. if it allignes with skills and competencies outlined by the provided CCSS standard itself.

Follow these steps to complete the task:
1. Understand the provided CCSS definition, skills and competencies it should test
2. Understand the expected CONTEXT complexity, lenght and topic
3. Validate if CONTEXT broadly meets the requirements
4. Validate if FRQ broadly meets the requirements and is relevant to the CONTEXT

If the CONTEXT and FRQ meet or mostly meet the CCSS requirements you will output "True". Otherwise output "False". 
In case you respond "False" also provide:
1. Justification what is missing or incorrect.
2. Adjusted CONTEXT and FRQ, that will be based on the existing, but enhanced to align better to the provided CCSS and user topic.
Remember: If the CONTEXT or FRQ doesnt meet the CCSS requirements, you must adjust both to keep the FRQ relevant to the CONTEXT.

{fmt_instr_frq_qa}
"""

In [67]:
# Make SystemMessagePromptTemplate
prompt=PromptTemplate(
    template=system_message,
    input_variables=["fmt_instr_frq_qa"]
)

system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)

In [68]:
CCSS_FRQ_QA = """Common Core State Standard is {standard}. Topic is {topic}. 
CONTEXT paragraph to be evaluated and Free Response Question (FRQ) to be evaluated are in the below JSON: 
{reasoning_context_frq}"""

In [69]:
# Make HumanMessagePromptTemplate
human_message_prompt = HumanMessagePromptTemplate.from_template(CCSS_FRQ_QA)

In [70]:
# Create ChatPromptTemplate: Combine System + Human
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat_prompt

ChatPromptTemplate(input_variables=['fmt_instr_frq_qa', 'reasoning_context_frq', 'standard', 'topic'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['fmt_instr_frq_qa'], output_parser=None, partial_variables={}, template='\nYou are an helpful assistant. User will provide you Common Core State Standard (CCSS) and a topic. You will also be provided with CONTEXT paragraph and Free Response Question (FRQ).\nYou will evaluate if provided CONTEXT and FRQ are meeting the CCSS requirements, i.e. if it allignes with skills and competencies outlined by the provided CCSS standard itself.\n\nFollow these steps to complete the task:\n1. Understand the provided CCSS definition, skills and competencies it should test\n2. Understand the expected CONTEXT complexity, lenght and topic\n3. Validate if CONTEXT broadly meets the requirements\n4. Validate if FRQ broadly meets the requirements and is relevant to the CONTEXT\n\nIf the CON

In [71]:
chain3 = LLMChain(
    llm=ChatOpenAI(
        temperature=0.0, 
        openai_api_base=OPENAI_API_BASE,
        openai_api_key=OPENAI_API_KEY,
        deployment_id = OPENAI_DEPLOYMENT_ID),
    prompt = chat_prompt,
    output_key="frq_qa"
)

WARNING! deployment_id is not default parameter.
                    deployment_id was transferred to model_kwargs.
                    Please confirm that deployment_id is what you intended.


In [83]:
# # DEBUG
# chain3
# with no_ssl_verification():
#     response = chain3.predict(standard = "CCSS.ELA-LITERACY.W.4.9", 
#                               topic = "desert",
#                               fmt_instr_frq_qa=fmt_instr_frq_qa,
#                               frq=output_parser_frq.parse(solution['reasoning_context_frq'])['frq'],
#                               context=output_parser_frq.parse(solution['reasoning_context_frq'])['context'])

# Chain 4 - QA Rubrics

In [84]:
# Define response schema
rubric_qa_json = ResponseSchema(name="rubric_qa_passed",
                             description="Boolean value to idicate if the RUBRIC is meeting the Common Core State Standard provided by user. True = meets, False = Does not meet")
rubric_qa_reason_json = ResponseSchema(name="rubric_qa_reason",
                             description="Justification why RUBRICS isnt meeting Common Core State Standard. If the Rubric is meeting CCSS provide value N/A")
rubric_qa_new_json = ResponseSchema(name="rubric_qa_new",
                             description="New regenerated Rubrics meeting Common Core State Standard. If the original Rubric is meeting CCSS provide value N/A")

response_schemas = [rubric_qa_json,rubric_qa_reason_json,rubric_qa_new_json]
output_parser_rubric_qa = StructuredOutputParser.from_response_schemas(response_schemas)
fmt_instr_rubric_qa = output_parser_rubric_qa.get_format_instructions()
print(fmt_instr_rubric_qa)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"rubric_qa_passed": string  // Boolean value to idicate if the RUBRIC is meeting the Common Core State Standard provided by user. True = meets, False = Does not meet
	"rubric_qa_reason": string  // Justification why RUBRICS isnt meeting Common Core State Standard. If the Rubric is meeting CCSS provide value N/A
	"rubric_qa_new": string  // New regenerated Rubrics meeting Common Core State Standard. If the original Rubric is meeting CCSS provide value N/A
}
```


In [85]:
# # to avoid Lanchain considering JSON definition as variable
# fmt_instr_rubric_qa_var = "{fmt_instr_rubric_qa}"

In [86]:
system_message = """
You are an helpful assistant. User will provide you Common Core State Standard (CCSS).\
You will also be provided with RUBRICS related to the CCSS.
You will evaluate if provided RUBRICS alignes CCSS requirements, i.e. if it allignes with skills and competencies outlined by the provided CCSS standard itself.
If the RUBRICS meets or mostly meets the CCSS requirements you will output "True". Otherwise output "False". 
In case you respond "False" also provide:
1. Justification what is missing or incorrect.
2. Adjusted Rubrics, that will be based on the existing, but enhanced to align better to the provided CCSS.
{fmt_instr_rubric_qa}
"""

In [87]:
# Make SystemMessagePromptTemplate
prompt=PromptTemplate(
    template=system_message,
    input_variables=["fmt_instr_rubric_qa"]
)

system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)

In [88]:
CCSS_RUBRIC_QA = """Common Core State Standard is {standard}. Rubrics to be evaluated is in JSON format: 
{rubric}."""

In [89]:
# Make HumanMessagePromptTemplate
human_message_prompt = HumanMessagePromptTemplate.from_template(CCSS_RUBRIC_QA)

In [90]:
# Create ChatPromptTemplate: Combine System + Human
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat_prompt

ChatPromptTemplate(input_variables=['fmt_instr_rubric_qa', 'rubric', 'standard'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['fmt_instr_rubric_qa'], output_parser=None, partial_variables={}, template='\nYou are an helpful assistant. User will provide you Common Core State Standard (CCSS).You will also be provided with RUBRICS related to the CCSS.\nYou will evaluate if provided RUBRICS alignes CCSS requirements, i.e. if it allignes with skills and competencies outlined by the provided CCSS standard itself.\nIf the RUBRICS meets or mostly meets the CCSS requirements you will output "True". Otherwise output "False". \nIn case you respond "False" also provide:\n1. Justification what is missing or incorrect.\n2. Adjusted Rubrics, that will be based on the existing, but enhanced to align better to the provided CCSS.\n{fmt_instr_rubric_qa}\n', template_format='f-string', validate_template=True), additional_kwargs={}),

In [91]:
chain4 = LLMChain(
    llm=ChatOpenAI(
        temperature=0.0, 
        openai_api_base=OPENAI_API_BASE,
        openai_api_key=OPENAI_API_KEY,
        deployment_id = OPENAI_DEPLOYMENT_ID),
    prompt = chat_prompt,
    output_key="rubics_qa"
)

WARNING! deployment_id is not default parameter.
                    deployment_id was transferred to model_kwargs.
                    Please confirm that deployment_id is what you intended.


In [92]:
# # DEBUG
# chain4
# with no_ssl_verification():
#     response = chain4.predict(standard = "CCSS.ELA-LITERACY.W.4.9", fmt_instr_rubric_qa=fmt_instr_rubric_qa, rubric=solution['rubric'])

# Overall Chain

In [93]:
overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["standard","topic","fmt_instr_rubric","fmt_instr_frq","fmt_instr_frq_qa","fmt_instr_rubric_qa","EXAMPLE_FRQ_01","EXAMPLE_FRQ_02","EXAMPLE_RUBRIC_01","EXAMPLE_RUBRIC_02"],
    output_variables=["rubric","reasoning_context_frq","frq_qa","rubics_qa"],
    verbose=True
)

with no_ssl_verification():
    solution = overall_chain({"standard":"CCSS.ELA-LITERACY.W.4.9", 
                              "topic":"desert", 
                              "fmt_instr_rubric":fmt_instr_rubric, 
                              "fmt_instr_frq":fmt_instr_frq,
                              "fmt_instr_frq_qa":fmt_instr_frq_qa,
                              "fmt_instr_rubric_qa":fmt_instr_rubric_qa, 
                              "EXAMPLE_FRQ_01":EXAMPLE_FRQ_01, 
                              "EXAMPLE_FRQ_02":EXAMPLE_FRQ_02,
                              "EXAMPLE_RUBRIC_01":EXAMPLE_RUBRIC_01, 
                              "EXAMPLE_RUBRIC_02":EXAMPLE_RUBRIC_02})

[chain/start] [1:chain:SequentialChain] Entering Chain run with input:
{
  "standard": "CCSS.ELA-LITERACY.W.4.9",
  "topic": "desert",
  "fmt_instr_rubric": "The output should be a markdown code snippet formatted in the following schema, including the leading and trailing \"```json\" and \"```\":\n\n```json\n{\n\t\"rubric\": string  // Rubric for this assesment task as expected by the Common Core State Standard definition provided by user\n}\n```",
  "fmt_instr_frq": "The output should be a markdown code snippet formatted in the following schema, including the leading and trailing \"```json\" and \"```\":\n\n```json\n{\n\t\"step-1\": string  // step 1 reasoning\n\t\"step-2\": string  // step 2 reasoning\n\t\"step-3\": string  // step 3 reasoning\n\t\"context\": string  // generated context paragraph\n\t\"frq\": string  // generated free respoce question\n}\n```",
  "fmt_instr_frq_qa": "The output should be a markdown code snippet formatted in the following schema, including the leading

[llm/end] [1:chain:SequentialChain > 2:chain:LLMChain > 3:llm:ChatOpenAI] [16.09s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{\n\t\"step-1\": \"The Common Core State Standard CCSS.ELA-LITERACY.W.4.9 states that 4th-grade students should 'Draw evidence from literary or informational texts to support analysis, reflection, and research.'\",\n\t\"step-2\": \"The assessment will test the student's ability to extract information from a context paragraph related to deserts. The student should then use this information to support their analysis or reflection in response to a specific free-response question.\",\n\t\"step-3\": \"For a 4th-grade level, the context paragraph should be between 100-150 words. It should contain a mix of factual and interpretive details, providing a well-rounded basis for answering the free-response question.\",\n\t\"context\": \"Deserts are characterized by their dry climate and lack of vegetation. They can be found all

In [94]:
solution

{'standard': 'CCSS.ELA-LITERACY.W.4.9',
 'topic': 'desert',
 'fmt_instr_rubric': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"rubric": string  // Rubric for this assesment task as expected by the Common Core State Standard definition provided by user\n}\n```',
 'fmt_instr_frq': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"step-1": string  // step 1 reasoning\n\t"step-2": string  // step 2 reasoning\n\t"step-3": string  // step 3 reasoning\n\t"context": string  // generated context paragraph\n\t"frq": string  // generated free respoce question\n}\n```',
 'fmt_instr_frq_qa': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"frq_qa_passed": string  // Boolean value to i

# Export output samples

In [96]:
# import json
# with open("transcripts\sample_01_CCSS.ELA-LITERACY.W.4.9.json", "w") as file:
#     json.dump(solution, file)